In [1]:
import gymnasium as gym
from stk_actor.wrappers import StuckStopWrapper
import torch
import torch.nn.functional as F

path = 'stk_actor/trained_agents/'
agent = 'normed_a2c_num5_best'

class PreprocessObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        """
        A Gym wrapper to preprocess mixed observation space (continuous + discrete)
        into a flat tensor.
        
        Args:
            env: The Gym environment to wrap.
        """
        super().__init__(env)
        self.observation_space = self._get_flat_observation_space(env.observation_space)
        self.mean = torch.load(path+f'{agent}/buffer_mean', map_location='cpu')
        self.std = torch.load(path+f'{agent}/buffer_std', map_location='cpu')

    def _get_flat_observation_space(self, observation_space):
        """
        Create a flat observation space based on the original observation space.
        
        Args:
            observation_space: Original observation space with 'continuous' and 'discrete' components.
        
        Returns:
            A flattened observation space.
        """
        continuous_dim = observation_space['continuous'].shape[0]
        discrete_dims = sum(space.n for space in observation_space['discrete'])
        flat_dim = continuous_dim + discrete_dims
        return gym.spaces.Box(low=-float('inf'), high=float('inf'), shape=(flat_dim,), dtype=float)

    def observation(self, obs):
        """
        Process the observation into a flat tensor.
        
        Args:
            obs: The raw observation from the environment.
        
        Returns:
            A preprocessed flat tensor.
        """
        continuous_obs, discrete_obs = obs['continuous'], obs['discrete']
        continuous_tensor = torch.FloatTensor(continuous_obs)
        
        discrete_tensors = [
            F.one_hot(torch.tensor(x), num_classes=num_classes.n).float()
            for x, num_classes in zip(discrete_obs, self.env.observation_space['discrete'])
        ]
        
        flat_tensor = torch.cat([continuous_tensor] + discrete_tensors)
        normed_flat_tensor = (flat_tensor - self.mean) / (self.std + 1e-8)
        return normed_flat_tensor

import gymnasium as gym
from gymnasium import Wrapper

class SkipFirstNStepsWrapper(Wrapper):
    def __init__(self, env, n):
        super().__init__(env)
        self.n = n

    def reset(self, **kwargs):
        # Reset the environment
        obs, info = self.env.reset(**kwargs)
        # Skip the first n steps
        for _ in range(self.n):
            obs, _, done, truncated, info = self.env.step(self.env.action_space.sample())
            if done or truncated:
                obs, info = self.env.reset(**kwargs)
        return obs, info

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from typing import Dict, List, Tuple, Union, Type
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gymnasium as gym
from gymnasium import spaces

def get_device(device: Union[torch.device, str] = "auto") -> torch.device:
    if device == "auto":
        device = "cuda"
    device = torch.device(device)
    if device.type == torch.device("cuda").type and not torch.cuda.is_available():
        return torch.device("cpu")
    return device

class BaseFeaturesExtractor(nn.Module):
    def __init__(self, observation_space: gym.Space, features_dim: int = 0) -> None:
        super().__init__()
        assert features_dim > 0
        self._observation_space = observation_space
        self._features_dim = features_dim
    @property
    def features_dim(self) -> int:
        return self._features_dim

def get_flattened_obs_dim(observation_space: spaces.Space) -> int:
    if isinstance(observation_space, spaces.MultiDiscrete):
        return sum(observation_space.nvec)
    else:
        return spaces.utils.flatdim(observation_space)

class FlattenExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space) -> None:
        super().__init__(observation_space, get_flattened_obs_dim(observation_space))
        self.flatten = nn.Flatten()
    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.flatten(observations)
    
class MlpExtractor(nn.Module):
    def __init__(
        self,
        feature_dim: int,
        net_arch: Union[List[int], Dict[str, List[int]]],
        activation_fn: Type[nn.Module],
        device: Union[torch.device, str] = "auto",
    ) -> None:
        super().__init__()
        # device = torch.get_device(device)
        policy_net: List[nn.Module] = []
        value_net: List[nn.Module] = []
        last_layer_dim_pi = feature_dim
        last_layer_dim_vf = feature_dim

        if isinstance(net_arch, dict):
            pi_layers_dims = net_arch.get("pi", []) 
            vf_layers_dims = net_arch.get("vf", []) 
        else:
            pi_layers_dims = vf_layers_dims = net_arch
        for curr_layer_dim in pi_layers_dims:
            policy_net.append(nn.Linear(last_layer_dim_pi, curr_layer_dim))
            policy_net.append(activation_fn())
            last_layer_dim_pi = curr_layer_dim
        for curr_layer_dim in vf_layers_dims:
            value_net.append(nn.Linear(last_layer_dim_vf, curr_layer_dim))
            value_net.append(activation_fn())
            last_layer_dim_vf = curr_layer_dim

        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf
        self.policy_net = nn.Sequential(*policy_net)#.to(device)
        self.value_net = nn.Sequential(*value_net)#.to(device)

    def forward(self, features: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        :return: latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features: torch.Tensor) -> torch.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: torch.Tensor) -> torch.Tensor:
        return self.value_net(features)

    
class Policy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn,):
        super().__init__()
        self.features_extractor = FlattenExtractor(observation_space)
        self.pi_features_extractor = self.features_extractor
        self.vf_features_extractor = self.features_extractor
        self.mlp_extractor = MlpExtractor(
            self.features_extractor.features_dim,
            net_arch=net_arch,
            activation_fn=activation_fn,
        )
        self.action_net = nn.Linear(net_arch[-1], sum(action_dims))
        self.value_net = nn.Linear(net_arch[-1], 1)


class UnifiedSACPolicy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn):
        super().__init__()
        
        self.shared = Policy(
            observation_space,
            action_dims,
            net_arch=net_arch,
            activation_fn=activation_fn
        )
        self.action_dims = action_dims
    
    def forward(self, x):
        x = self.shared.features_extractor(x)
        x = self.shared.mlp_extractor.policy_net(x)
        x = self.shared.action_net(x)
        return x
    
    def sample(self, x, deterministic=False):
        logits = self.forward(x)
        
        # Split logits for each action dimension
        split_logits = torch.split(logits, self.action_dims, dim=-1)
        
        actions = []
        log_probs = []
        probs = []
        
        for logit in split_logits:
            distribution = Categorical(logits=logit)
            if deterministic:
                action = torch.argmax(logit, dim=-1)
            else:
                action = distribution.sample()
            
            log_prob = distribution.log_prob(action)
            prob = F.softmax(logit, dim=-1)
            
            actions.append(action)
            log_probs.append(log_prob)
            probs.append(prob)
        
        return (
            torch.stack(actions),
            torch.stack(log_probs),
            probs
        )
    
#policy = torch.load('policy_512_512_512_512_SiLU_3_statedict', map_location='cuda')


from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
import gymnasium as gym
from pystk2_gymnasium import AgentSpec
from bbrl.agents.gymnasium import ParallelGymAgent, make_env
from functools import partial

tracks = [
    'abyss',
    'black_forest',
    'candela_city',
    'cocoa_temple',
    'cornfield_crossing',
    'fortmagma',
    'gran_paradiso_island',
    'hacienda',
    'lighthouse',
    'mines',
    'minigolf',
    'olivermath',
    'ravenbridge_mansion',
    'sandtrack',
    'scotland',
    'snowmountain',
    'snowtuxpeak',
    'stk_enterprise',
    'volcano_island',
    'xr591',
    'zengarden',

# # #   ==================   #

#     'fortmagma',
#     'ravenbridge_mansion',
#     'snowmountain',
#     'cocoa_temple',
#     'sandtrack',    
#     'scotland', 
#     'stk_enterprise',
#     'volcano_island', # 1104
#     'xr591', # 864   
]

# karts = [4,12]
karts = [3,12]
n_envs = len(tracks)*len(karts)

print('making', n_envs, 'environments')
vec_env = make_vec_env(
    "supertuxkart/flattened_multidiscrete-v0",
    # seed=12,
    n_envs=n_envs, 
    wrapper_class=lambda x : (
        SkipFirstNStepsWrapper(
            StuckStopWrapper(
                PreprocessObservationWrapper(x),
                n=128,
            ), 
            n=19,
        )
    ), 
    env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"), #'track':'minigolf', 
    'laps':1,
    'difficulty':2, 
    'num_kart':12, #'difficulty':0
})

ix = 0
for num_kart in enumerate(karts):
    for track in enumerate(tracks):
        venv = vec_env.envs[ix]
        venv.env.default_track = track
        venv.env.num_kart = num_kart
        print(ix, track, )
        ix+=1



net_arch=[1024,1024,1024]
activation_fn=torch.nn.Tanh
# filename = 'policy_normed_1024_1024_1024_Tanh_statedict_2'
filename = path+f'{agent}/statedict'

action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename, map_location='cpu'))


making 42 environments
..:: Antarctica Rendering Engine 2.0 ::..


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_33918/2491321862.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mean = torch.load(path+f'{agent}/buffer

..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
0 (0, 'abyss')
1 (1, 'black_forest')
2 (2, 'candela_city')
3 (3, 'cocoa_temple')
4 (4, 'cornfield_crossing')
5 (5, 'fortmagma')
6 (6, 'gran_paradiso_island')
7 (7, 'hacienda')
8 (8, 'lighthouse')
9 (9, 'mines')
10 (10, 'minigolf')
11 (11, 'olivermath')
12 (12, 'ravenbridge_mansion')
13 (13, 'sandtrack')
14 (14, 'scotland')
15 (15, 'snowmountain')
16 (16, 'snowtuxpeak')
17 (17, 'stk_enterprise')
18 (18, 'volcano_island')
19 (19, 'xr591')
20 (20, 'zengarden')
21 (0, 'abyss')
22 (1, 'black_forest')
23 (2, 'candela_city')
24 (3, 'cocoa_temple')
25 (4, 'cornfield_crossing')
26 (5, 'fortmagma')
27 (6, 'gran_paradiso_island')
28 (7, 'hacienda')
29 (8, 'lighthouse

/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_33918/2491321862.py:327: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unified_policy.load_state_dict(torch.load(f

<All keys matched successfully>

..:: Antarctica Rendering Engine 2.0 ::..


In [ ]:
from stable_baselines3 import A2C


steps = [(
    # 1024*8,
    5*n_envs,
    1_000_000,
)]
for n_steps, total_timesteps in steps:
    # model = PPO(
    #     "MlpPolicy", 
    #     vec_env, 
    #     verbose=1, 
    #     policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
    #     device='cpu',
    #     learning_rate=0.0001,
    #     batch_size=128,
    #     n_epochs=100,
    #     n_steps=n_steps,
    #     tensorboard_log="./outputs/",
    #     # ent_coef=0.001,
    #     clip_range=0.2,
    # )
    model = A2C(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
        device='cpu',
        # learning_rate=0.001,
        n_steps=n_steps,
        tensorboard_log="./outputs/",
        use_rms_prop=False,
        normalize_advantage=True,
    )
    print('DOING', n_steps, total_timesteps)
    model.policy.load_state_dict(unified_policy.shared.state_dict())
    model.learn(total_timesteps=total_timesteps, progress_bar=True, log_interval=1)

        


..:: Antarctica Rendering Engine 2.0 ::..
Using cpu device
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica R

Output()

----------------------------
| time/              |     |
|    fps             | 9   |
|    iterations      | 1   |
|    time_elapsed    | 21  |
|    total_timesteps | 210 |
----------------------------


------------------------------------
| time/                 |          |
|    fps                | 16       |
|    iterations         | 2        |
|    time_elapsed       | 24       |
|    total_timesteps    | 420      |
| train/                |          |
|    entropy_loss       | -0.125   |
|    explained_variance | 0.428    |
|    learning_rate      | 0.0007   |
|    n_updates          | 1        |
|    policy_loss        | -0.0792  |
|    value_loss         | 151      |
------------------------------------


------------------------------------
| time/                 |          |
|    fps                | 22       |
|    iterations         | 3        |
|    time_elapsed       | 28       |
|    total_timesteps    | 630      |
| train/                |          |
|    entropy_loss       | -0.184   |
|    explained_variance | 0.397    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2        |
|    policy_loss        | -0.00817 |
|    value_loss         | 107      |
------------------------------------


...

------------------------------------
| time/                 |          |
|    fps                | 57       |
|    iterations         | 47       |
|    time_elapsed       | 171      |
|    total_timesteps    | 9870     |
| train/                |          |
|    entropy_loss       | -0.322   |
|    explained_variance | 0.924    |
|    learning_rate      | 0.0007   |
|    n_updates          | 46       |
|    policy_loss        | -0.0167  |
|    value_loss         | 44.3     |
------------------------------------


------------------------------------
| time/                 |          |
|    fps                | 57       |
|    iterations         | 48       |
|    time_elapsed       | 175      |
|    total_timesteps    | 10080    |
| train/                |          |
|    entropy_loss       | -0.292   |
|    explained_variance | 0.924    |
|    learning_rate      | 0.0007   |
|    n_updates          | 47       |
|    policy_loss        | 0.0129   |
|    value_loss         | 40.9     |
------------------------------------


------------------------------------
| time/                 |          |
|    fps                | 57       |
|    iterations         | 49       |
|    time_elapsed       | 178      |
|    total_timesteps    | 10290    |
| train/                |          |
|    entropy_loss       | -0.263   |
|    explained_variance | 0.965    |
|    learning_rate      | 0.0007   |
|    n_updates          | 48       |
|    policy_loss        | 0.0296   |
|    value_loss         | 19.7     |
------------------------------------


------------------------------------
| time/                 |          |
|    fps                | 57       |
|    iterations         | 50       |
|    time_elapsed       | 181      |
|    total_timesteps    | 10500    |
| train/                |          |
|    entropy_loss       | -0.314   |
|    explained_variance | 0.956    |
|    learning_rate      | 0.0007   |
|    n_updates          | 49       |
|    policy_loss        | 0.0241   |
|    value_loss         | 25.4     |
------------------------------------


------------------------------------
| time/                 |          |
|    fps                | 57       |
|    iterations         | 51       |
|    time_elapsed       | 184      |
|    total_timesteps    | 10710    |
| train/                |          |
|    entropy_loss       | -0.293   |
|    explained_variance | 0.848    |
|    learning_rate      | 0.0007   |
|    n_updates          | 50       |
|    policy_loss        | -0.019   |
|    value_loss         | 88.2     |
------------------------------------


In [ ]:
model.save(f'final_a2c_{n_steps}')
f'final_a2c_{n_steps}'

'final_a2c_2048'